In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("main_database_for_models.csv")

In [3]:
df.head(2)

,season,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,aPos,aGSPG,aGCPG,aCSPG,...,aGD7,hPosLS,aPosLS,hSpent,hIncome,hNet,aSpent,aIncome,aNet,target
0,9,4.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,...,0.0,18,18,18.72,0.479,-18.241,18.07,1.32,-16.75,2
1,9,5.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,5,9,47.15,34.830,-12.320,8.92,38.16,29.24,2


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [5]:
X_train.head(2)

,season,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,aPos,aGSPG,aGCPG,aCSPG,...,aGD5,aGD7,hPosLS,aPosLS,hSpent,hIncome,hNet,aSpent,aIncome,aNet
6206,25,4.0,2.08,1.00,0.33,2.08,10.0,1.17,1.33,0.33,...,0.6,0.571,2,16,96.3,9.32,-86.98,36.62,22.94,-13.68
40,9,9.0,1.25,1.75,0.00,1.50,7.0,1.67,1.00,0.33,...,0.0,0.000,14,3,17.1,23.65,6.55,47.84,10.92,-36.92


In [6]:
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[0]

array([ 1.75041495e-01,  2.80066393e-02,  1.45634524e-02,  7.00165982e-03,
        2.31054774e-03,  1.45634524e-02,  7.00165982e-02,  8.19194199e-03,
        9.31220756e-03,  2.31054774e-03,  8.75207477e-03,  9.97736524e-02,
        3.26767464e-02,  8.40199178e-02,  2.56750865e-02,  1.00123735e-03,
        5.81137765e-04,  1.16717669e-02,  1.26029877e-02,  1.50045570e-02,
        7.00165982e-03,  5.60132785e-03,  8.00289717e-03,  7.00165982e-03,
        9.80232374e-03,  1.09996076e-02, -4.67010710e-03,  4.20099589e-03,
        3.99794776e-03,  1.40033196e-02,  1.12026557e-01,  6.74259840e-01,
        6.52554695e-02, -6.09004371e-01,  2.56400782e-01,  1.60618076e-01,
       -9.57827063e-02])

In [7]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(64, activation = tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [8]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1.688,
                1: 1.747,
                2: 1}

In [9]:
model.fit(X_train_norm, y_train.values, epochs=50, validation_data=(X_test_norm,y_test), class_weight=class_weight)

Train on 5168 samples, validate on 1292 samples
Epoch 1/50
5168/5168 [==============================] - 2s 343us/step - loss: 1.0670 - acc: 0.4449 - val_loss: 1.0510 - val_acc: 0.4621
Epoch 2/50
5168/5168 [==============================] - 1s 192us/step - loss: 1.0405 - acc: 0.4847 - val_loss: 1.0336 - val_acc: 0.4868
Epoch 3/50
5168/5168 [==============================] - 1s 181us/step - loss: 1.0330 - acc: 0.4789 - val_loss: 1.0154 - val_acc: 0.5008
Epoch 4/50
5168/5168 [==============================] - 1s 186us/step - loss: 1.0293 - acc: 0.4847 - val_loss: 1.0127 - val_acc: 0.5000
Epoch 5/50
5168/5168 [==============================] - 1s 191us/step - loss: 1.0273 - acc: 0.4843 - val_loss: 1.0047 - val_acc: 0.5039
Epoch 6/50
5168/5168 [==============================] - 1s 190us/step - loss: 1.0231 - acc: 0.4876 - val_loss: 1.0585 - val_acc: 0.4567
Epoch 7/50
5168/5168 [==============================] - 2s 323us/step - loss: 1.0218 - acc: 0.4938 - val_loss: 1.0359 - val_acc: 0.4698


In [10]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1,2]))

preds: {0: 2486, 1: 1096, 2: 1586}
real: {0: 1444, 1: 1322, 2: 2402}
[[1093  586  807]
 [ 192  442  462]
 [ 159  294 1133]]


In [11]:
print(conf_matrix(test_pred,y_test.values))

[[244 169 205]
 [ 53  78 132]
 [ 64  83 264]]
